## Retrieving the Dataset
1. Change runtime type to `Python 3 with GPU`
2. Authorize the notebook to mount your Google Drive
3. Decompress the Dataset onto the local disk (Ephemeral Storage of the provisioned Google Colab VM)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!cat /content/gdrive/My\ Drive/DashGC/20bn-jester-v1-?? | tar -zx

In [ ]:
import os

print(os.getcwd())
dirs = os.listdir("20bn-jester-v1")
print(len(dirs))

directories = {dir: len(os.listdir("20bn-jester-v1" + "/" + dir)) for dir in dirs}
print(len(directories.keys()))
print(min(directories.values()), max(directories.values()))